# Taaltheorie & Taalverwerking 2018 - Assignment 6

In this assignment, we will work more with WordNet and explore music processing with NLTK tools.

In [11]:
# FILL THIS IN FOR YOUR GROUP, also name your file as: tttv_ass6_<group>_<name1>_<name2>.ipynb
# Group        : D
# Name - UvaID : 11430087
# Name - UvaID :
# Date         : 30-05-2018

Don't forget to load WordNet!

In [2]:
from math import *
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/willem/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Question 1 (6pts total)

In this exercise you will explore the output of a distributional semantic model and compare its semantic similarity ranking to that obtained with path-length distance in an ontology. 

#### Question 1.1 (1pt)
Infomap is an implementation of a distributional semantic model that can be queried online at http://clic.cimec.unitn.it/infomap-query/. For a target word, it will return its $n$ nearest semantic neighbours ordered by similarity strength (calculated using cosine similarity between vectors in semantic space). Have a look at the model options that can be selected in the querying interface. Read the Infomap documentation (link at the end of the  page) and briefly summarise (in your own words) the features of the model option **bnc-lemma-narrow**. Your summary should include the language for which the model has been constructed, the corpus used, the characteristics of the target words (the rows), and the size of the context.

#### Answers:

The bnc-lemma-narrow's has been trained on an english corpus of only nouns, verbs and adjectives. 
The corpus had all of its original words replaced with their corresponding lemma's. Furthermore, 
there were exactly 20000 target words with 2000 features. The context in which the target words occured 
were composed of sentences of only five words long with the target word occuring only in the middle.


#### Question 1.2 (2pts)
Search for the target word **car** using the following parameters: target word PoS **noun**, model option **bnc-lemma-narrow**, neighbor PoS  **noun**, max. number of neighbors **5**. As output you should get a list of five words including **car**, with a similarity score that indicates how similar they are to **car** according to the model. 

Use NLTK to determine the lowest common hypernym of these five words in *WordNet*.

**Hint:** Don't forget that you need to check all possible senses for each word. The lowest common hypernym is the lowest-level synset that subsumes at least one sense of all five words. You probably *don't* want to use the NLTK **lowest_common_hypernyms()** method. (Can you see why?)

In [53]:
# Could have gone for something more efficient,
# but this works more than fine and is simple.

search_list = []

car = wn.synsets('car')
van = wn.synsets('van')
vehicle = wn.synsets('vehicle')
truck = wn.synsets('truck')
motorcycle = wn.synsets('motorcycle')

search_list.extend(car)
search_list.extend(van)
search_list.extend(vehicle)
search_list.extend(truck)
search_list.extend(motorcycle)

# returns common lowest hypernym for a list of synsets
def BFS (search_list):
    while search_list:
        element = search_list.pop(0)
        search_list.extend(element.hypernyms())
        hypos = set(s for s in element.closure(lambda s:s.hyponyms()))
        if goal(hypos):
            return element
        
    return False

# true if hyponyms are present in all given sets
def goal(hypos):
    if (    not set.isdisjoint(hypos, set(car)) \
        and not set.isdisjoint(hypos, set(van)) \
        and not set.isdisjoint(hypos, set(vehicle)) \
        and not set.isdisjoint(hypos, set(truck)) \
        and not set.isdisjoint(hypos, set(motorcycle))):
        return True
        
    
print(BFS(search_list)) # lowest common hypernym is: 'conveyance.n.03'

Synset('conveyance.n.03')


#### Question 1.3 (2pts)

Implement a function called **co\_hyponym(node1, node2)** to check whether two *words* (not synsets) are co-hyponyms or sister terms (i.e., whether they have some immediate hypernym in common). Which of the other words on your Infomap list are co-hyponyms of 'car'?

In [49]:
# Check if they have the same parent.

def co_hyponym(word1, word2):
    co_hypos = []
    word1_synsets = wn.synsets(word1)
    word2_synsets = wn.synsets(word2)
    
    if not word1_synsets or not word2_synsets:
        print('Input not found')
        return 

    for word1 in word1_synsets:
        word1_hypers = set(word1.hypernyms())
        for word2 in word2_synsets:
            if not word1_hypers.isdisjoint(set(word2.hypernyms())):
                co_hypos.append((word1, word2))
    if co_hypos == []:
        return False
    else:
        return co_hypos
            
            

print(co_hyponym("car", "van")) # not co-hyponyms
print(co_hyponym("car", "vehicle")) # not co-hyponyms
print(co_hyponym("car", "truck")) # co-hyponyms
print(co_hyponym("car", "motorcycle")) # co-hyponyms 

#print(wn.synsets('motorcyle'))

False
False
[(Synset('car.n.01'), Synset('truck.n.01'))]
[(Synset('car.n.01'), Synset('motorcycle.n.01'))]


#### Question 1.3 (1pt)
Give the results of **textbook_similarity** queries between **car** and each of the other four words in this ontology, using your function from last week. Give the resulting list of semantic neighbours of **car** ordered by similarity strength (according to WordNet path-length) and compare this ranking to the one output by Infomap.


In [30]:
def textbook_similarity(word1, word2, pos):
    if word1 == word2:
        print("inf")
        return ""
    
    word1_synsets = wn.synsets(word1, pos = pos)
    word2_synsets = wn.synsets(word2, pos = pos)

    #print('Word1 synsets: {}'.format(word1_synsets))
    #print('Word2 synsets: {}'.format(word2_synsets))
    for synset1 in word1_synsets:
      if synset1 in word2_synsets:
        print('Early return, similarity is infinity')
        return

    from itertools import chain
    def is_similar(hypernyms1, hypernyms2):
      similarities = []
      for i in range(len(hypernyms1)):
        for j in range(len(hypernyms2)):
          for nym in hypernyms2[j]:
            if nym in hypernyms1[i]:
              return (i + j)
      return None

    nyms1 = word1_synsets
    nyms2 = word2_synsets
    word1_hypernyms = []  # 2D array
    word2_hypernyms = []  # 2D array
    while (
            (
              len(list(chain.from_iterable(nym.hypernyms() for nym in nyms1))) > 0 
              or len(list(chain.from_iterable(nym.hypernyms() for nym in nyms2))) > 0 
            )
          ):
      nyms1 = list(chain.from_iterable(nym.hypernyms() for nym in nyms1))
      nyms2 = list(chain.from_iterable(nym.hypernyms() for nym in nyms2))
      word1_hypernyms.append(nyms1)
      word2_hypernyms.append(nyms2)
      #print('At depth {}'.format(max([len(word1_hypernyms), len(word2_hypernyms)])))

    if is_similar([word1_synsets]+word1_hypernyms, [word2_synsets]+word2_hypernyms) is not None:
        print('{}'.format(is_similar([word1_synsets]+word1_hypernyms, [word2_synsets]+word2_hypernyms)))
    else:
        print('-inf')
    return ""

print(textbook_similarity("car", "van", pos = wn.NOUN))
print(textbook_similarity("car", "vehicle", pos = wn.NOUN))
print(textbook_similarity("car", "truck", pos = wn.NOUN))
print(textbook_similarity("car", "motorcycle", pos = wn.NOUN))

# The ranking is quite similar to the one given by infomap.
# They mayor difference is that infomap doesn't rank vehicle, 
# truck and motorcycle as equally similar, in contrary
# to the textbook similarity.

1

2

2

2



## Question 2: Parsing Eurovision (6pts total)

This question will introduce the basics of grammar-based syntactic analysis of musical harmony.

### Question 2.1 (1pt)

Go to http://chordify.net and run an analysis of Australia's official entry for the Eurovision Song Contest 2018: https://www.youtube.com/watch?reload=9&v=J4XZxbrvepw. Copy the first six chords down as they appear in the Chordify interface, as a list of Python strings. You may find it slightly faster if you change to the *Akkorden* view instead of *Diagrammen*.

**Answer:**

In [ ]:
australian_chords = ["G", "E", "Em", "C", "D", "G"]

### Question 2.2 (2pts)

The following grammar is a (somewhat simplified) implementation of the harmonic grammar Fred Lerdahl proposes in his book *Tonal Pitch Space* (2001). In addition to the traditional harmonic classes of tonic, dominant, and subdominant harmony, Lerdahl's grammar includes a *departure* class (`Dep`), a *return* class (`Ret`), and a *neighbour* class (`N`). His definitions of the harmonic classes in terms of Roman numerals are commented out: the 'lexical rules' for this grammar.

The tonal centre of Australia's Eurovision entry is G. Use the principles we discussed in class to replace the Roman-numeral lexical rules with lexical rules for all of the chord symbols in `australian_chords` (e.g., `T -> 'G'`).

Run the code block when you are finished to see how many parse trees Lerdahl's grammar can compute for this sentence.

**Hint:** If you still find the rules for converting between chord names and Roman numerals confusing, try browsing Wikipedia's surprisingly good collection of articles on harmony, starting here: https://en.wikipedia.org/wiki/Roman_numeral_analysis

In [ ]:
from nltk import CFG
from nltk.parse.chart import LeftCornerChartParser

lerdahl_grammar = CFG.fromstring("""
  P -> T
  T -> T T
  T -> D T
  D -> D D
  D -> S D
  S -> S S
  T -> T N T
  D -> D N D
  S -> S N S
  T -> T Dep
  D -> D Dep
  S -> S Dep
  Dep -> N Dep
  T -> Ret T
  D -> Ret D
  S -> Ret S
  
  # Replace the following strings with the actual chord symbols you need.
  # 
  # T -> 'I'
  # D -> 'V' | 'VII'
  # S -> 'II' | 'III' | 'IV' | 'VI' | 'VII'
  # Dep -> 'II' | 'III' | 'IV' | 'V' | 'VI' | 'VII' 
  # Ret -> 'II' | 'III' | 'IV' | 'V' | 'VI' | 'VII' 
  # N -> 'II' | 'III' | 'IV' | 'V' | 'VI' | 'VII' 
""")

lerdahl_parser = LeftCornerChartParser(lerdahl_grammar)
lerdahl_parses = lerdahl_parser.parse(australian_chords)

# Print the total number of parses as well as the actual parse trees.
lerdahl_sum = 0
for t in lerdahl_parses: 
    lerdahl_sum = lerdahl_sum + 1
    t.pretty_print()
print(lerdahl_sum, 'trees')

### Question 2.3 (2pts)

The next grammar is based on the foundations of Chordify (Bas de Haas, *Music Information Retrieval Based on Tonal Harmony*, 2012). Chordify uses a different set of harmonic classes: tonic, dominant, subdominant, and *tonic prolongation* (`TPG`). They also define the members of the traditional classes differently than Fred Lerdahl; theirs  are based on a beautiful branch of music theory known as neo-Riemannian theory. Replace the commented 'lexical rules' in this grammar with actual chord symbols from `australian_chords`, just like you did in the previous question, and run the block.

In [ ]:
chordify_grammar = CFG.fromstring("""
  P -> PCPa
  P -> PCPb
  P -> HCP
  P -> P P

  PCPa -> D T | D D T
  PCPa -> PCPa T
  PCPb -> T D T
  HCP -> T D
  HCP -> T HCP
  D -> S D
  T -> TPG
  
  T -> T T
  D -> D D
  S -> S S
  
  # Replace the following strings with the actual chord symbols you need.
  # 
  # T -> 'I'
  # D -> 'V' | 'VII'
  # S -> 'II' | 'IV' 
  # TPG -> 'III' | 'VI'
""")

chordify_parser = LeftCornerChartParser(chordify_grammar)
chordify_parses = chordify_parser.parse(australian_chords)

# Print the total number of parses as well as the actual parse trees.
chordify_sum = 0
for t in chordify_parses: 
    chordify_sum = chordify_sum + 1
    t.pretty_print()
print(chordify_sum, 'trees')

### Question 2.4 (1pt):

Which grammar is more practical for daily use, Lerdahl's or Chordify's? Why?

**Answer:**